# Tools

This notebook contains all the tools that will be used by the agent.

In [29]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location 

The following function look up at latitude and longitude of a city.

In [6]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master
df = pd.read_csv('data/cities_latlng.csv', sep=';')
df.head()
df.columns = ['id', 'country', 'city', 'latitude', 'longitude', 'altitude']
df.drop('country', axis=1, inplace=True)
df.head()

,id,city,latitude,longitude,altitude
0,2,Kandahar,31.610000,65.699997,1015
1,3,Mazar-e Sharif,36.706944,67.112221,369
2,4,Herat,34.340000,62.189999,927
3,5,Jalalabad,34.420000,70.449997,573
4,6,Konduz,36.720000,68.860001,394


In [7]:
# TODO: Explore the loaded dataframe
df[df['city'] == 'Singapore']

,id,city,latitude,longitude,altitude
10566,10568,Singapore,1.29027,103.851959,164


In [24]:
# TODO: Add tool description

@tool
def get_latlng(city: str) -> any:
   """
   Return the latitude, longitude and altitude of a city in dictionary 

   Args:
      city: the of of the city that you want the latitude, longitude and altitude

   Returns:
      any: a dictionary with the following keys: city, latitude, longitude, altitude

   Example:
      result = get_latlng('singapore')
   """
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   return { 'city': city, 'latitude': r.iloc[0]['latitude'], 'longitude': r.iloc[0]['longitude'], 'altitude': r.iloc[0]['altitude'] }

In [22]:
# TODO: Test get_latlng method
# case insensitive search
get_latlng('tokyo')

{'city': 'tokyo',
 'latitude': np.float64(35.6895266),
 'longitude': np.float64(139.6916809),
 'altitude': np.int64(40)}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [25]:
# TODO: Add tool description

@tool
def get_temperature(latitude: float, longitude: float) -> any:
   """ 
   Get the temperature of a location given by its latitude and longitude

   Args:
      latitude: latitude as float
      longitude: longitude as float

   Returns:
      any: a dictionary with the following keys: temperature_unit, temperature

   Example:
      result = get_temperature(35.6895266, 139.6916809)

   """
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }

In [23]:
# TODO: Test get_temperature method
get_temperature(35.6895266, 139.6916809)

{'temperature_unit': '°C', 'temperature': 26.0}

In [27]:
# TODO: Create a web search tool

def web_search(query: str) -> any:

   search = DuckDuckGoSearchRun()
   return search.invoke(query)

In [31]:
%pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.3 MB/s eta 0:00:0031m12.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [duckduckgo_search]
Note: you may need to restart the kernel to use updated packages.


In [32]:
# TODO: Test the web search
web_search('Cast of Andor season 2')

"Meet the stars of the live-action Star Wars series, from Cassian Andor himself, Diego Luna, to new faces and returning Rogue One favorites. Learn about their characters, the journey so far, and the excitement for the road ahead in Andor Season 2. Meet the actors and characters of Andor season 2, the Star Wars TV show that bridges Rogue One and A New Hope. Learn about Cassian Andor, Luthen Rael, Mon Mothma, Bix Caleen, and more. Many of the cast members from Season 1 return, including Adria Arjona as Andor's love interest Bix Caleen, Stellan Skarsgard as ruthless spymaster Luthen Rael, Genevieve O'Reilly as Senator ... Find out who will return and join the Rebel Alliance in Andor season 2, the Star Wars spin-off series on Disney+. Diego Luna, Stellan Skarsgard, Adria Arjona, Ben Mendelsohn, and more will reprise their roles from season 1 and Rogue One. Orson Krennic, the character played by Ben Mendelsohn was absent in Andor Season 1 but will be a part of season 2. He was cast for the 

### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------| 
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [ ]:
# TODO: Add tool description

@tool
def query_album_track(query: str) -> any:
   """ 
   Perform SQL queries on the album_track table. Return the result as an array of records.
   The table name is album_track and has the following columns:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name: nvarchar(120)
      duration: integer
      composer: nvarchar(220)
   The duration is in milliseconds.
   Use this for query only. 

   Args:
      query: a valid SQL query

   Return
      any: list of tuple. Each element corresponds to a record 

   Example:
      results = query_album_track("select * from album_track where artist_name like '%michael%'")
   """
   database = "data/chinook_sqlite.sqlite" 
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor() 
      rows = cursor.execute(query)
      return rows.fetchall()
   finally:
      conn.close()

In [37]:
# TODO: Test the query_album_track function
query_album_track('select count(*) from album_track')


[(3503,)]

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [40]:
class SQLiteTool(Tool):

   # what is the name of this tool
   name = "album track query"

   description = """ 
   Perform SQL queries on the album_track table. Return the result as an array of records.
   The table name is album_track and has the following columns:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name: nvarchar(120)
      duration: integer
      composer: nvarchar(220)
   The duration is in milliseconds.
   Use this for query only. 
   """

   # JSON schema - https://json-schema.org/
   inputs = {
      "query": {
         "type": "string",
         "description": "A valid SQL query"
      }
   }

   output_type = "list"

   def __init__(self, db_file):
      self.db_file = db_file 
      self.setup()

   def forward(self, query: str) -> list:
      conn = sqlite3.connect(self.db_file)
      try:
         cursor = conn.cursor() 
         rows = cursor.execute(query)
         return rows.fetchall()
      finally:
         conn.close()   